In [2]:
import numpy as np
import pandas as pd
import os
import math
from scipy.signal import butter, lfilter, freqz
import matplotlib.pyplot as plt

In [129]:
paths = []

for i in range(1, 20):
    root, file = os.path.split("../Dataset/SA01/D01_SE15_R01.csv") #바꿀곳 파일명만
    if i < 10:
        file = file[:2] + str(i) + file[3:]
        tmp = os.path.join(root, file)
        paths.append(tmp)
        if i > 4:
            for j in range(2, 6):
                file_new = file[:-5] + str(j) + file[-4:]
                tmp = os.path.join(root, file_new)
                paths.append(tmp)
    if i >= 10:
        file = file[:1] + str(i) + file[3:]
        for j in range(1, 6):
            file = file[:-5] + str(j) + file[-4:]
            tmp = os.path.join(root, file)
            paths.append(tmp)

paths[4:14] = ['../Dataset/SA01/D05_SA01_R01_1.csv',
                '../Dataset/SA01/D05_SA01_R01_2.csv',
                '../Dataset/SA01/D05_SA01_R02_1.csv',
                '../Dataset/SA01/D05_SA01_R02_2.csv',
                '../Dataset/SA01/D05_SA01_R03_1.csv',
                '../Dataset/SA01/D05_SA01_R03_2.csv',
                '../Dataset/SA01/D05_SA01_R04_1.csv',
                '../Dataset/SA01/D05_SA01_R04_2.csv',
                '../Dataset/SA01/D05_SA01_R05_1.csv',
                '../Dataset/SA01/D05_SA01_R05_2.csv',
                '../Dataset/SA01/D06_SA01_R01_1.csv',
                '../Dataset/SA01/D06_SA01_R01_2.csv',
                '../Dataset/SA01/D06_SA01_R02_1.csv',
                '../Dataset/SA01/D06_SA01_R02_2.csv',
                '../Dataset/SA01/D06_SA01_R03_1.csv',
                '../Dataset/SA01/D06_SA01_R03_2.csv',
                '../Dataset/SA01/D06_SA01_R04_1.csv',
                '../Dataset/SA01/D06_SA01_R04_2.csv',
                '../Dataset/SA01/D06_SA01_R05_1.csv',
                '../Dataset/SA01/D06_SA01_R05_2.csv']

paths[-15:-10] = ['../Dataset/SA01/D17_SA01_R01_1.csv',
                 '../Dataset/SA01/D17_SA01_R01_2.csv',
                 '../Dataset/SA01/D17_SA01_R02_1.csv',
                 '../Dataset/SA01/D17_SA01_R02_2.csv',
                 '../Dataset/SA01/D17_SA01_R03_1.csv',
                 '../Dataset/SA01/D17_SA01_R03_2.csv',
                 '../Dataset/SA01/D17_SA01_R04_1.csv',
                 '../Dataset/SA01/D17_SA01_R04_2.csv',
                 '../Dataset/SA01/D17_SA01_R05_1.csv',
                 '../Dataset/SA01/D17_SA01_R05_2.csv']

SE15_D_paths = [] #바꿀 곳 2

for path in paths:
    path = path.replace('SA01', 'SE15') # 바꿀곳 3 (바꿀것, 바뀔것)
    SE15_D_paths.append(path) # 바꿀 곳 4

SE15_D_paths # 바꿀 곳 5

['../Dataset/SE15\\D01_SE15_R01.csv',
 '../Dataset/SE15\\D02_SE15_R01.csv',
 '../Dataset/SE15\\D03_SE15_R01.csv',
 '../Dataset/SE15\\D04_SE15_R01.csv',
 '../Dataset/SE15/D05_SE15_R01_1.csv',
 '../Dataset/SE15/D05_SE15_R01_2.csv',
 '../Dataset/SE15/D05_SE15_R02_1.csv',
 '../Dataset/SE15/D05_SE15_R02_2.csv',
 '../Dataset/SE15/D05_SE15_R03_1.csv',
 '../Dataset/SE15/D05_SE15_R03_2.csv',
 '../Dataset/SE15/D05_SE15_R04_1.csv',
 '../Dataset/SE15/D05_SE15_R04_2.csv',
 '../Dataset/SE15/D05_SE15_R05_1.csv',
 '../Dataset/SE15/D05_SE15_R05_2.csv',
 '../Dataset/SE15/D06_SE15_R01_1.csv',
 '../Dataset/SE15/D06_SE15_R01_2.csv',
 '../Dataset/SE15/D06_SE15_R02_1.csv',
 '../Dataset/SE15/D06_SE15_R02_2.csv',
 '../Dataset/SE15/D06_SE15_R03_1.csv',
 '../Dataset/SE15/D06_SE15_R03_2.csv',
 '../Dataset/SE15/D06_SE15_R04_1.csv',
 '../Dataset/SE15/D06_SE15_R04_2.csv',
 '../Dataset/SE15/D06_SE15_R05_1.csv',
 '../Dataset/SE15/D06_SE15_R05_2.csv',
 '../Dataset/SE15\\D07_SE15_R01.csv',
 '../Dataset/SE15\\D07_SE15_R0

In [137]:
# def butter_lowpass(cutoff, fs, order=4):
#     nyq = 0.5 * fs
#     normal_cutoff = cutoff / nyq
#     b, a = butter(order, normal_cutoff, btype='low', analog=False)
#     return b, a

# def butter_lowpass_filter(data, cutoff, fs, order=4):
#     b, a = butter_lowpass(cutoff, fs, order=order)
#     x = np.trunc(np.array(data, dtype=np.float64)).astype(np.int64)
#     y = lfilter(b, a, x, axis=-1, zi=None)
#     return y

# # Setting standard filter requirements.
# order = 4
# fs = 200
# cutoff = 5

# b, a = butter_lowpass(cutoff, fs, order)

# # Creating the data for filteration
# T = 200         # value taken in seconds
# n = int(T * fs) # indicates total samples
# t = np.linspace(0, T, n, endpoint=False)

In [130]:
list = []
LPF_list = []
final_SE15_D = pd.DataFrame(list)  # 바꿀곳 6


for path in SE15_D_paths: # 바꿀곳 7
    try:
        data = pd.read_csv(path, header=None)
        data = data.iloc[:, :3]
        rows, cols = data.shape

        sliced_df = data[0 : 160]
    
        # LPF 구하기

#         col_1_lpf = butter_lowpass_filter(sliced_df[0].to_numpy(), cutoff, fs, order).astype('float64')
#         col_2_lpf = butter_lowpass_filter(sliced_df[1].to_numpy(), cutoff, fs, order).astype('float64')
#         col_3_lpf = butter_lowpass_filter(sliced_df[2].to_numpy(), cutoff, fs, order).astype('float64')
    
    
        col_1_std = np.sqrt(np.absolute(np.mean(sliced_df[0]) - sliced_df[0])**2 / len(sliced_df[0]))
        col_2_std = np.sqrt(np.absolute(np.mean(sliced_df[1]) - sliced_df[1])**2 / len(sliced_df[1]))
        col_3_std = np.sqrt(np.absolute(np.mean(sliced_df[2]) - sliced_df[2])**2 / len(sliced_df[2]))
    
        # 가속도 구하기
        # 표준편차 구하기

        col_1 = col_1_std*col_1_std
        col_2 = col_2_std*col_2_std
        col_3 = col_3_std*col_3_std
    
        C_nine = np.sqrt(col_1 + col_2 + col_3)
        list_C_nine = pd.DataFrame(C_nine).transpose().round()
        final_SE15_D = final_SE15_D.append(list_C_nine) # 바꿀곳 8

    except:
        pass

pd.set_option('display.max_row', 500)
pd.set_option('display.max_columns', 100)

row, col = final_SE15_D.shape  #바꿀곳 9
target_col = row * [0]
final_SE15_D.loc[:,'target'] = target_col # 바꿀곳 10

final_SE15_D

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,...,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,target
0,4.0,4.0,5.0,5.0,6.0,6.0,6.0,6.0,6.0,5.0,5.0,4.0,3.0,3.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,3.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,8.0,7.0,6.0,6.0,6.0,7.0,7.0,7.0,7.0,7.0,6.0,6.0,6.0,6.0,6.0,6.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,4.0,3.0,2.0,1.0,1.0,1.0,2.0,2.0,3.0,3.0,4.0,4.0,4.0,5.0,5.0,5.0,5.0,5.0,4.0,4.0,5.0,4.0,4.0,4.0,5.0,5.0,0
0,8.0,8.0,8.0,7.0,6.0,6.0,6.0,5.0,6.0,6.0,6.0,7.0,7.0,8.0,8.0,9.0,9.0,9.0,9.0,8.0,7.0,6.0,5.0,4.0,3.0,3.0,3.0,4.0,4.0,5.0,6.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,6.0,5.0,5.0,5.0,4.0,4.0,5.0,6.0,6.0,7.0,8.0,8.0,...,8.0,7.0,6.0,5.0,6.0,6.0,6.0,6.0,6.0,6.0,5.0,5.0,5.0,5.0,5.0,4.0,5.0,4.0,4.0,3.0,3.0,2.0,2.0,3.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,9.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,9.0,9.0,9.0,9.0,8.0,8.0,7.0,8.0,7.0,7.0,0
0,17.0,17.0,14.0,10.0,7.0,6.0,7.0,8.0,10.0,11.0,13.0,15.0,16.0,17.0,18.0,17.0,17.0,16.0,15.0,14.0,13.0,11.0,10.0,9.0,8.0,8.0,7.0,7.0,7.0,7.0,6.0,5.0,4.0,3.0,3.0,3.0,3.0,4.0,4.0,5.0,5.0,6.0,6.0,6.0,6.0,7.0,7.0,7.0,7.0,7.0,...,6.0,7.0,8.0,9.0,9.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,11.0,12.0,13.0,14.0,15.0,16.0,17.0,17.0,18.0,18.0,17.0,17.0,16.0,15.0,14.0,15.0,16.0,18.0,19.0,21.0,20.0,16.0,13.0,13.0,16.0,19.0,21.0,24.0,26.0,27.0,26.0,24.0,22.0,0
0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,4.0,3.0,3.0,3.0,2.0,3.0,2.0,2.0,2.0,2.0,2.0,2.0,3.0,2.0,2.0,2.0,2.0,2.0,3.0,2.0,2.0,2.0,3.0,3.0,3.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,3.0,3.0,3.0,3.0,4.0,4.0,4.0,5.0,5.0,5.0,6.0,6.0,6.0,6.0,6.0,6.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,6.0,6.0,6.0,6.0,6.0,6.0,7.0,7.0,7.0,7.0,7.0,7.0,8.0,8.0,8.0,0
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0
0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,3.0,3.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,3.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,3.0,3.0,4.0,4.0,4.0,5.0,6.0,6.0,7.0,7.0,8.0,8.0,8.0,8.0,9.0,8.0,8.0,8.0,0
0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,3.0,3.0,4.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,4.0,4.0,4.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,...,2.0,2.0,1.0,1.0,2.0,2.0,2.0,2.0,3.0,3.0,4.0,4.0,4.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,4.0,4.0,4.0,3.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,2.0,0
0,3.0,3.0,3.0,3.0,3.0,3.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,4.0,3.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,...,3.0,3.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,3.0,3.0,3.0,3.0,3.0,3.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,6.0,5.0,0
0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,

In [15]:
A01 = final_SA01_D #변수지정(사람번호) - 다시해야함

In [18]:
A02 = final_SA02_D 

In [22]:
A03 = final_SA03_D

In [25]:
A04 = final_SA04_D

In [28]:
A05 = final_SA05_D

In [31]:
A06 = final_SA06_D

In [34]:
A07 = final_SA07_D

In [37]:
A08 = final_SA08_D

In [40]:
A09 = final_SA09_D

In [43]:
A10 = final_SA10_D

In [46]:
A11 = final_SA11_D

In [49]:
A12 = final_SA12_D

In [52]:
A13 = final_SA13_D

In [55]:
A14 = final_SA14_D

In [58]:
A15 = final_SA15_D

In [61]:
A16 = final_SA16_D

In [64]:
A17 = final_SA17_D

In [67]:
A18 = final_SA18_D

In [70]:
A19 = final_SA19_D

In [73]:
A20 = final_SA20_D

In [76]:
A21 = final_SA21_D

In [79]:
A22 = final_SA22_D

In [82]:
A23 = final_SA23_D

In [85]:
E01 = final_SE01_D

In [88]:
E02 = final_SE02_D

In [91]:
E03 = final_SE03_D

In [94]:
E04 = final_SE04_D

In [97]:
E05 = final_SE05_D

In [100]:
E06 = final_SE06_D

In [103]:
E07 = final_SE07_D

In [106]:
E08 = final_SE08_D

In [109]:
E09 = final_SE09_D

In [112]:
E10 = final_SE10_D

In [115]:
E11 = final_SE11_D

In [118]:
E12 = final_SE12_D

In [121]:
E13 = final_SE13_D

In [127]:
E14 = final_SE14_D

In [131]:
E15 = final_SE15_D

In [132]:
ADL_all = pd.concat([A01, A02, A03, A04, A05, A06, A07, A08, A09, A10, A11, A12, A13, A14, A15, A16, A17, A18, A19, A20, A21, A22, A23, E01, E02, E03, E04, E05, E06, E07, E08, E09, E10, E11, E12, E13, E14, E15])

In [134]:
ADL_all.shape

(3210, 161)

In [133]:
ADL_all.to_csv('[C9]ADL_all_noLPF.csv',index=False)